In [ ]:
using Revise
using Plots
pyplot()
# plotly()

using QSimulator

# Transmon Ramsey Oscillations

Starting from a coherence between two energy eigenstates we expect oscilltions at the detuning between the energy levels.  We'll work with natural units of GHz and ns for numerical stability reasons. We'll start with a single 3 level transmon in the lab frame with a 5 GHz qubit frequency and -200 MHz anharmonicity.

In [ ]:
# we create a specific QSystem
q0 = FixedDuffingTransmon("q0", 5, -0.2, 3)
# we can ask for the Hamiltonian of an QSystem as a Matrix
hamiltonian(q0)

In [ ]:
# as CompositeQSystems is a tensor product structure of QSystem's and is what all the solvers are built around
cqs = CompositeQSystem([q0]);
add_hamiltonian!(cqs, hamiltonian(q0), "q0")
hamiltonian(cqs)

In [ ]:
# evolve an initial superpositions state for 1 ns
times = collect(linspace(0,1,201))
ψ0 = (1/sqrt(2)) * Complex128[1; 1; 0]
ψs = unitary_state(cqs, times, ψ0);

In [ ]:
# plot the projection on to the initial state and we expect to see 5 GHz oscillations
signal = Float64[real(ψ0'*ψ) for ψ in ψs]
expected = 0.5 + 0.5*cos.(2π*5 * (linspace(0,1,201)))
p = plot(times, signal, linewidth=2, label="simulated")
plot!(p, times, expected, label="ideal")
xlabel!(p, "Time (ns)")

# Rabi Oscillations

Driving Rabi oscillations in the lab frame is a good example of a parametric time dependent Hamiltonian. The drive electric field couples to the transmon dipole or $X$ operator.

## Constant Drive

In [ ]:
function qubit_drive(q::QSimulator.QSystem, drive::Function)
    function add_drive_ham!(ham, idxs, t)
        pulse = 2π * drive(t)
        drive_ham = real(pulse) * X(q) + imag(pulse) * Y(q)
        QSimulator.expand_add!(ham, drive_ham, idxs)
    end
end

qubit_freq = 5.0
q0 = FixedDuffingTransmon("q0", qubit_freq, -0.2, 3)
cqs = CompositeQSystem([q0]);
add_hamiltonian!(cqs, hamiltonian(q0), "q0")
add_hamiltonian!(cqs, qubit_drive(q0, t -> 0.02*cos(2π*qubit_freq * t)), q0);
ψ_init = Complex128[1; 0; 0]
times = collect(linspace(0,100,101))
ψs = unitary_state(cqs, times, ψ_init);

In [ ]:
p = plot(times, [abs2(s[1]) for s in ψs], label="Ground State Pop.")
plot!(p, times, [abs2(s[2]) for s in ψs], label="Excited State Pop.")
xlabel!(p, "Time (ns)")

## Variable Amplitude Gaussian Pulse

In [ ]:
# write a helper function that returns the drive Hamiltonian at a particular point in time
function gaussian(pulse_length, pulse_freq, t; cutoff=2.5)
    σ = pulse_length/2/cutoff
    pulse = exp(-0.5*((t-pulse_length/2)/σ)^2)
    pulse * cos(2π*pulse_freq * t)
end

function flat(pulse_freq,  t)
   cos(2π*pulse_freq * t)
end

function qubit_drive(qubit::QSimulator.QSystem, drive::Function)
    function add_drive_ham!(ham, idxs, t)
        pulse = 2π * drive(t)
        drive_ham = real(pulse) * X(qubit) + imag(pulse) * Y(qubit)
        QSimulator.expand_add!(ham, drive_ham, idxs)
    end
end

In [ ]:
ψ0 = Complex128[1; 0; 0]
states_flat = []
states_gaussian = []
amps = 0.05 * linspace(0,1, 51)
pulse_length = 50.0
qubit_freq = 5.0

for amp = amps
    # first do flat pulse
    # three level transmon in the lab frame
    q0 = FixedDuffingTransmon("q0", qubit_freq, -0.2, 3)
    cqs = CompositeQSystem([q0]);
    add_hamiltonian!(cqs, hamiltonian(q0), "q0")
    add_hamiltonian!(cqs, qubit_drive(q0, t -> amp*flat(qubit_freq, t)), q0);
    ψs = unitary_state(cqs, [0, pulse_length], ψ0)
    push!(states_flat, ψs[end])

    # now gaussian
    cqs = CompositeQSystem([q0]);
    add_hamiltonian!(cqs, hamiltonian(q0), "q0")
    add_hamiltonian!(cqs, qubit_drive(q0, t -> amp*gaussian(pulse_length, qubit_freq, t)), q0);
    ψs = unitary_state(cqs, [0, pulse_length], ψ0)
    push!(states_gaussian, ψs[end])
end

In [ ]:
p1 = plot(amps*1e3, [abs2(s[1]) for s in states_flat], label="Ground State Pop.")
plot!(p1, amps*1e3, [abs2(s[2]) for s in states_flat], label="Excited State Pop.")
plot!(p1, amps*1e3, [abs2(s[3]) for s in states_flat], label="Excited State Pop.")
xlabel!(p1, "Nutation Strength (MHz)")
title!(p1, "Flat Pulse")
p2 = plot(amps*1e3, [abs2(s[1]) for s in states_gaussian], label="Ground State Pop.")
plot!(p2, amps*1e3, [abs2(s[2]) for s in states_gaussian], label="Excited State Pop.")
plot!(p2, amps*1e3, [abs2(s[3]) for s in states_gaussian], label="Excited State Pop.")
xlabel!(p2, "Peak Nutation Strength (MHz)")
title!(p2, "Gaussian Pulse")
plot(p1,p2, layout=(1,2), size=(800,400))


# Two Qubit  Gates - Parametric Gates in the Lab Frame

In [ ]:
# helper function to add flux drive
function flux_drive(tt::QSimulator.QSystem, amp, freq)
    function add_drive_ham!(ham, idxs, t)
        tt_ham = 2π * hamiltonian(tt, amp*sin(2π * freq * t))
        QSimulator.expand_add!(ham, tt_ham, idxs)
    end
end

In [ ]:
# parameters from Blue Launch paper
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)


freqs = 117:0.25:127
times = collect(0.0:5:1000)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[1.0; 0.0; 0.0] # start in 10 state
pop_01 = []
pop_10 = []

for freq = freqs
    cqs = CompositeQSystem([q0, q1])
    add_hamiltonian!(cqs, hamiltonian(q0), q0)
    add_hamiltonian!(cqs, 0.006*Dipole(q0, q1), [q0,q1])
    add_hamiltonian!(cqs, flux_drive(q1, 0.323, freq/1e3), q1)
    ψs = unitary_state(cqs, times, ψ0);
    push!(pop_01, [abs2(ψ[2]) for ψ in ψs])
    push!(pop_10, [abs2(ψ[4]) for ψ in ψs])
end

In [ ]:
p1 = contour(freqs, times, cat(2, pop_01...), fill=true)
p2 = contour(freqs, times, cat(2, pop_10...), fill=true)
xlabel!(p2, "Frequency (MHz)")
plot(p1,p2, layout=(2,1), size=(600,1000))

In [ ]:
# Look more finely at a slice along time to show we are getting full contrast
q0 = FixedDuffingTransmon("q0", 3.94015, -0.1807,  3)
q1 = TunableDuffingTransmon("q1",  0.172, 16.4, 0.55, 3)


# should get an iSWAP interaction at ≈ 122 MHz
freq = 122.1
times = collect(0.0:0.5:200)
ψ0 = Complex128[0.0; 1.0; 0.0] ⊗ Complex128[1.0; 0.0; 0.0] # start in 10 state

cqs = CompositeQSystem([q0, q1])
add_hamiltonian!(cqs, hamiltonian(q0), q0)
add_hamiltonian!(cqs, 0.006*Dipole(q0, q1), [q0,q1])
add_hamiltonian!(cqs, flux_drive(q1, 0.323, freq/1e3), q1)
ψs = unitary_state(cqs, times, ψ0);
pop_01 = [abs2(ψ[2]) for ψ in ψs]
pop_10 = [abs2(ψ[4]) for ψ in ψs];

In [ ]:
p = plot(times, pop_01, label="01")
plot!(p, times, pop_10, label="10")